**DATA CLEANER**

**Generating a pandas dataframe from the source .csv file**

In [91]:
import pandas as pd
import seaborn as sns
import numpy as np

# Set options to show all columns
pd.set_option('display.max_columns', None)

# Load csv file and generate pandas dataframe
data_scrape_csv = r'..\data\raw\immo_scraper_merged_with_landsurface.csv' # Fill your path to file
data_scraped = pd.read_csv(data_scrape_csv, sep = ',')


**Adding columns on price per m^2**
- And display of the info of the dataframe at start of analysis

In [92]:
# Adding new column of price per m2 nethabitable surface with safeguard for ZeroDivisionError
data_scraped['price_per_sqm'] = np.where(data_scraped['Living_area'] == 0, np.nan, data_scraped['Price'] / data_scraped['Living_area'])

# Adding new column of price per m2 land surface with safeguard for ZeroDivisionError
data_scraped['price_per_sqm_land'] = np.where(data_scraped['Living_area'] == 0, np.nan, data_scraped['Price'] / data_scraped['landSurface'])

# Display info on the dataframe
data_scraped.info()
data_scraped_shape = data_scraped.shape


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9995 entries, 0 to 9994
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  9995 non-null   int64  
 1   locality_name       9995 non-null   object 
 2   Postal_code         9995 non-null   object 
 3   Price               9340 non-null   float64
 4   Subtype             9995 non-null   object 
 5   Number_of_rooms     3328 non-null   float64
 6   Number_of_bedrooms  9356 non-null   float64
 7   Living_area         9170 non-null   float64
 8   sale_annuity        16 non-null     object 
 9   Type_of_sale        9995 non-null   object 
 10  street              9700 non-null   object 
 11  number              9589 non-null   object 
 12  latitude            9705 non-null   float64
 13  longitude           9705 non-null   float64
 14  Open_fire           9995 non-null   bool   
 15  Swimming_Pool       3859 non-null   object 
 16  hasTer

**Cosmetic clean and preprocessing:**
- cosmetic changes (e.g. no blank spaces)
- text fields as strings, and in title case
- no wrongly encoded values (e.g. a text value in the price column, numeric values as integer or float)

In [93]:
# Copy dataframe contents to a new dataframe
data_clean_1 = data_scraped.copy()

# Edit text in the columns
text_edit_columns = ['Subtype', 'Type_of_sale', 'Kitchen_type', 'State_of_building' ]

for column in text_edit_columns:
    data_clean_1[column] = data_clean_1[column].astype(str)  # Ensure the column is treated as string
    data_clean_1[column] = data_clean_1[column].str.replace('_', ' ').str.capitalize() #Replace "_" to space, editing text (first capital letter, rest lower case)

#Edit text of cities and street names
names_edit_columns = ['locality_name', 'street']

for column in names_edit_columns:
    data_clean_1[column] = data_clean_1[column].astype(str)  # Ensure the column is treated as string
    data_clean_1[column] = data_clean_1[column].str.title() # Editing text (first capital letters, rest lower case)

# Remove zip code from brackets, e.g. "Tielt (8700)" -> "Tielt"
data_clean_1['locality_name'] = data_clean_1['locality_name'].str.replace(r"\s*\(\d+\)", "", regex=True)

# Edit the numbers in columns

# Selecting the columns to change
number_edit_columns = ['Number_of_bedrooms', 'Living_area', 'Number_of_facades', 'Price', 'terraceSurface', 'gardenSurface']

# Converting the cell value to int (so that they are integers), if there is no value in the cell - fill in 0.
for column in number_edit_columns:
    data_clean_1[column] = pd.to_numeric(data_clean_1[column], errors='coerce').fillna(0).astype('Int64')

# Values conversion cells from false to 0, true to 1.
columns_for_change_01 = ['Open_fire', 'Swimming_Pool', 'Furnished', 'hasTerrace'] # Columns for change
data_clean_1[columns_for_change_01] = data_clean_1[columns_for_change_01].replace({False:0, True:1}).astype('Int64')

# Display first 5 rows
data_clean_1.head()
data_clean_1_shape = data_clean_1.shape


C:\Users\tom_v\AppData\Local\Temp\ipykernel_9832\2092513498.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_clean_1[columns_for_change_01] = data_clean_1[columns_for_change_01].replace({False:0, True:1}).astype('Int64')


**Cleanups involving assumptions on missing values**
- 'Open_fire', 'Swimming_pool', 'Furnished', 'hasTerrace': if missing values for --> set to 0
- 'hasGarden', 'epc', 'Kitchen_type', 'State_of_building': missing values --> set to 0

In [94]:
# Copy dataframe contents to a new dataframe
data_clean_2 = data_clean_1.copy()

# Replace empty cells value on binary values with 0
columns_for_change_01 = ['Open_fire', 'Swimming_Pool', 'Furnished', 'hasTerrace'] # Columns for change

# Values conversion cells from false to 0, true to 1, if empty fill as 0.
data_clean_2[columns_for_change_01] = data_clean_2[columns_for_change_01].fillna(0).astype('Int64')

# Replace "0" and "Nan" on "Null"

# Columns for change
columns_for_change_null = ['hasGarden', 'epc', 'Kitchen_type', 'State_of_building']

# Replace 0 and Nan on 'null'
data_clean_2[columns_for_change_null] = data_clean_2[columns_for_change_null].replace({0: 'null', 'Nan': 'null'})

# Empty cells fill as 'null'
data_clean_2[columns_for_change_null] = data_clean_2[columns_for_change_null].fillna('null')

# Display first 5 rows
data_clean_2.head()
data_clean_2_shape = data_clean_2.shape

**Removal of records - based on field values:**
- removed duplicates
- removed records with price, living area, or nr of bedrooms = 0

In [95]:
# Copy dataframe contents to a new dataframe, via removal of duplicates
data_clean_3 = data_clean_2.drop_duplicates(subset=['latitude', 'longitude', 'street', 'number', 'Subtype'])
duplicates = data_clean_2[data_clean_2.duplicated(subset=['latitude', 'longitude', 'street', 'number', 'Subtype'])]

# Remove lines where 'Price' is = 0
count_price_zero = (data_clean_3['Price'] == 0).sum()
data_clean_3 = data_clean_3[data_clean_3['Price'] != 0]

# Remove lines where 'Living area' is = 0
count_living_area_zero = (data_clean_3['Living_area'] == 0).sum()
data_clean_3 = data_clean_3[data_clean_3['Living_area'] != 0]

# Remove lines where 'Number_of_bedrooms' is = 0
count_bedrooms_zero = (data_clean_3['Number_of_bedrooms'] == 0).sum()
data_clean_3 = data_clean_3[data_clean_3['Number_of_bedrooms'] != 0]

data_clean_3_shape = data_clean_3.shape

print("Number of duplicates:", len(duplicates))
print("Number with zero price:", count_price_zero)
print("Number with zero living area:", count_living_area_zero)
print("Number with zero bedrooms:", count_bedrooms_zero)

# Checks how many properties per subtype
data_clean_3['Subtype'].value_counts()

Number of duplicates: 905
Number with zero price: 563
Number with zero living area: 172
Number with zero bedrooms: 71


Subtype
House                   6188
Villa                    865
Mixed use building       310
Apartment block          239
Mansion                  161
Exceptional property     161
Town house               109
Bungalow                  91
Country cottage           90
Farmhouse                 38
Chalet                    13
Manor house               10
Castle                     8
Other property             1
Name: count, dtype: int64

**Removal of records - based on property subtypes:**
- removed records with starting price (indicates house groups or public sales)
- removed subtypes house group, castle, exceptional properties, 'other' properties --> these property types 'House group', 'Castle', 'Other property', 'Exceptional property' are not used in our analysis of housing and apartment prices because they represent categories that are either undefined ('other property'), not comparable ('house group'), or exceptional ('castle','expceptional property').
- removed chalet and bungalow, as they reflect a different segment, such as trailers on a camping, tiny houses, but also villa style properties in bad shape.

Our analysis focuses on more typical housing types to ensure relevance and consistency in the data. See also later for further focusing on the subtype 'House'.

In [97]:
# Copy dataframe contents to a new dataframe
data_clean_4 = data_clean_3.copy()

# Remove lines where 'Starting_price' is True
count_has_starting_price = (data_clean_4['Starting_price'] == True).sum()
data_clean_4 = data_clean_4[data_clean_4['Starting_price'] != True]

# Cleaning data from subtypes that are not our focus:

subtypes_to_remove = ['House group', 'Chalet', 'Bungalow', 'Castle', 'Other property', 'Exceptional property']
count_subtypes_removed = (data_clean_4['Subtype'].isin(subtypes_to_remove)).sum()
data_clean_4 = data_clean_4[~data_clean_4['Subtype'].isin(subtypes_to_remove)]

# Display info on the dataframe
data_clean_4_shape = data_clean_4.shape

# Checks how many properties per subtype
data_clean_4['Subtype'].value_counts()

Subtype
House                 5656
Villa                  802
Mixed use building     282
Apartment block        220
Mansion                155
Town house             101
Country cottage         82
Farmhouse               36
Manor house             10
Name: count, dtype: int64

**Removing unnecessary columns**

In [98]:
# Removing unnecessary columns
columns_to_drop = ['Number_of_rooms', 'sale_annuity', 'Starting_price', 'Type_of_sale', 'hasGarden']
data_clean_5 = data_clean_4.drop(columns=columns_to_drop, errors='ignore')

# Display info on the dataframe
data_clean_5_shape = data_clean_5.shape

**Saving data to a new .csv file and a .pkl file**

In [99]:
# Save data to new csv file

output_csv = r'..\data\clean\after_step_1_cleaning.csv'  # Fill your path to file
data_clean_5.to_csv(output_csv, index=False)

# Save data to new pkl file

import pickle
output_pkl= r'..\data\clean\after_step_1_cleaning.pkl' # Fill your path to file
with open(output_pkl, 'wb') as f:
    pickle.dump(data_clean_5, f)

**Summary of cleaning operation:**

In [102]:

print("Info on dataframe at start of the first cleaning step:")
data_scraped.info()
print("Dataframe shape after scraping: ", data_scraped_shape) 
print("Dataframe shape after cosmetic cleanup: ", data_clean_1_shape)
print("Dataframe shape after removal of records - based on assumptions on missing values: ", data_clean_2_shape)

print("Amount of duplicates: ", len(duplicates))
print("Amount with zero price: ", count_price_zero)
print("Amount with zero living area: ", count_living_area_zero)
print("Amount with zero bedrooms: ", count_bedrooms_zero)
print("Dataframe shape after removal of records - based on field values: ", data_clean_3_shape)

print("Amount removed with starting price: ", count_has_starting_price)
print("Amount removed on subtype: ", count_subtypes_removed)
print("Dataframe shape after removal of records - based on property subtypes: ", data_clean_4_shape)

print("Dataframe shape after removal of unnecessary columns: ", data_clean_5_shape)
print(f"The CSV file was saved as {output_csv} and pickled to {output_pkl}.")
print("Info on dataframe at the end of the first cleaning step:")
data_clean_5.info()

Info on dataframe at start of the first cleaning step:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9995 entries, 0 to 9994
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  9995 non-null   int64  
 1   locality_name       9995 non-null   object 
 2   Postal_code         9995 non-null   object 
 3   Price               9340 non-null   float64
 4   Subtype             9995 non-null   object 
 5   Number_of_rooms     3328 non-null   float64
 6   Number_of_bedrooms  9356 non-null   float64
 7   Living_area         9170 non-null   float64
 8   sale_annuity        16 non-null     object 
 9   Type_of_sale        9995 non-null   object 
 10  street              9700 non-null   object 
 11  number              9589 non-null   object 
 12  latitude            9705 non-null   float64
 13  longitude           9705 non-null   float64
 14  Open_fire           9995 non-null   bool   
 15  